# Visualizando a malha de serviços com Kiali

Discutiremos sobre muitas ferramentas durante este curso, mas o [Kiali](https://kiali.io/) é a ferramenta que ajudará você a entender e administrar sua malha de serviços.

Até a versão 1.5, o Kiali e outras ferramentas faziam parte da distribuição do Istio (_profile_), mas desde a versão 1.6 essas ferramentas devem ser instaladas separadamente.

Convenientemente o download do Istio, que fizemos na primeira parte, contém essas ferramentas.

Mas antes de instalar o Kiali, vamos verificar o acesso ao cluster:

In [ ]:
# [Opcional] Se você não estiver usando o docker-desktop será necessário obter o arquivo de configuração e ajustar a variável KUBECONFIG
#export KUBECONFIG=~/.kube/config # Local do kubeconfig do docker-desktop

kubectl config get-contexts
kubectl get nodes

> Se você estiver utilizando o docker-desktop ou minikube, provavelmente não precisará configurar a variável de ambiente _KUBECONFIG_ a cada lição, mas se receber algum erro de conexão com o cluster, verifique se a variável existe e está apontando para o arquivo de configuração correto (`echo $KUBECONFIG`).

> Você também pode passar a variável de ambiente para o Jupyter Lab no terminal: `env KUBECONFIG=~/.kube/config jupyter lab`. Isso fará com que ela esteja acessível em todas as lições (notebooks).

Ok, estamos prontos para continuar, mas se tiver algum problema, consulte "Acessando o cluster" na parte 1.

### Instalando o kiali e dependências no cluster

Para instalar o Kiali iremos aplicar o arquivo [kiali.yaml](istio-1.7.4/samples/addons/kiali.yaml), mas antes vamos inspecioná-lo, clique no link para abri-lo.

O arquivo kiali é composto de alguns recursos, são eles:

* CDR
* ServiceAccount
* ConfigMap
* ClusterRole
* ClusterRoleBinding
* Deployment
* Service
* MonitoringDashboard

Com exceção do `MonitoringDashboard`, todos os demais recursos são do kubernetes e o CRD (Custom Resource Definition) é uma forma de criar novos recursos no kubernetes, neste caso, ele define o `MonitoringDashboard`. Mais sobre kuberntes você pode obter no nosso curso [Kubernetes avançado para iniciantes](TODO), onde abordamos esses temas e muitos outros.

Vamos instalar e conhecer o kiali antes de nos aprofundar neste CDR.

Bug versão 1.7.0 a 1.9.1 - [Istio 1.7.1 unable to install Kiali addon #27417](https://github.com/istio/istio/issues/27417)

Necessário aplicar o CRD antes do restante dos recursos.

In [ ]:
cat <<EOF | kubectl apply -f -
apiVersion: apiextensions.k8s.io/v1beta1
kind: CustomResourceDefinition
metadata:
  name: monitoringdashboards.monitoring.kiali.io
spec:
  group: monitoring.kiali.io
  names:
    kind: MonitoringDashboard
    listKind: MonitoringDashboardList
    plural: monitoringdashboards
    singular: monitoringdashboard
  scope: Namespaced
  versions:
  - name: v1alpha1
    served: true
    storage: true
EOF

Agora podemos instalar os recursos do kiali:

In [ ]:
ISTIO_VERSION=1.9.1

kubectl apply -f istio-$ISTIO_VERSION/samples/addons/kiali.yaml

E o prometheus, responsável por coletar dados dos _containers_ que serão utilizados pelo kiali:

In [ ]:
kubectl apply -f istio-$ISTIO_VERSION/samples/addons/prometheus.yaml

Vamos verificar o que foi instalado:

In [ ]:
kubectl get all -n istio-system

O serviço do kiali é do tipo `ClusterIP`, o que significa que não podemos acessá-lo diretamente de fora do cluster, há algumas alternativas, modificar ou criar um serviço do tipo `NodePort` ou `LoadBalancer`, configurar um `Ingress` ou usar o subcomando `port-forward` do `kubectl`.

Porém o `istioctl` oferece um subcomando conveniente para acessar o kiali, o `istioctl dashboard kiali`.

Vamos acessa-lo, mas antes, vamos gerar algum tráfego para a nossa aplicação, copie conteúdo do arquivo [assets/scripts/simple-app.sh](assets/scripts/simple-app.sh) e cole em um terminal:

Agora vamos acessar o painel:

In [ ]:
istioctl dashboard kiali

E voilá, você está acessando o painel do kiali.

Vamos explorar alguns recursos do kiali.

![kiali overview](media/kiali-overview.gif)

O kiali oferece uma visibilidade inpressionante da malha de serviços e tudo que precisamos fazer é anotar o _namespace_ onde nossa aplicação será instalada.

Interrompa a execução do dashboard:
* No Jupyter Lab ou Notebook: Selecione a célula com o comando `istioctl dashboard kiali` e clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Se você participou do nosso curso de [Kubernetes avançado para iniciantes](TODO) deve imaginar o que o `istioctl`automatizou, foi o comando `kubectl port-forward` e adicionou um comando de ` open` para abrir a página inicial no navegador.

O comando a seguir tem efeito semelhante (sem a parte do navegador)

In [ ]:
kubectl port-forward service/kiali 20001:20001 -n istio-system

Kiali _dashboard_: <http://localhost:20001>

Mesmo resultado, mas não tão elegante. Ficaremos com o `istioctl dashboard <dashboard>` pelo resto do curso.

Interrompa a execução do `port-forward` do mesmo jeito que fizemos com o `istioctl`.

E vamos executar o kiali novamente, mas agora em segundo plano.

In [ ]:
istioctl dashboard kiali &
export KIALI_PID=$!

E acessá-lo pela url <http://localhost:20001>, caso não abra automaticamente.

### Labels

Para kiali, os rótulos `app` e `version` são reconhecidos automaticamente e utilizados para agrupar os serviços e as cargas de trabalho.

Esses dois rótulos são requeridos e o Kiali irá indicar um alerta caso algum deles esteja faltando, embora isso não afete o funcionamento da aplicação.

Podemos utilizar os rótulos para localizar os recurso no kuberentes:

In [ ]:
kubectl describe deploy -l app=simple-app

E no nosso [deployment](exemplos/2_simple-app/deployment.yaml) adicionamos na seção _template_ os rótulos que desejamos que sejam adicionados aos PODs que forem criados.

In [ ]:
kubectl get pods -l app=simple-app,version=v1

A sintaxe para busca de recursos utilizando rótulos é bem rica no kubernetes, para mais exemplos acesse [kubernetes - Labels and Selectors](https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/).

## Conclusão

O kiali não é uma ferramenta estática, uma das suas forças e exibir a relação entre os recursos em tempo de execução. Nas próximas seções iremos explorar uma aplicação com uma malha de serviços.

Você pode manter o redirecionamento para o dashboard do kiali ou pará-lo, iremos utilizá-lo várias vezes durante o curso. Então lembre-se de não executá-lo novamente se deixa-lo rodando.

Para interrompê-lo pare o processo do `istioctl`, que salvamos na variável `KIALI_PID`.

In [ ]:
kill $KIALI_PID

## Limpando

Não precisaremos mais da nossa aplicação de teste, vamos exluí-la para liberar recursos do cluster.

Os recursos criados podem ser obtidos passando as mesmas configurações que utilizamos para criá-los.

In [ ]:
kubectl get -f assets/exemplos/simple-app

E o mesmo vale para excluí-los.

> O kubectl não solicita confirmação para execução, tome cuidado e revise o comando antes de executá-lo.

In [ ]:
kubectl delete -f assets/exemplos/simple-app